In [1]:
# Importation
import numpy as np
import pandas as pd
import plotly.express as px

# **I - Chargement des données**

In [5]:
# Chargement des données
df = pd.read_csv("../data/01_251021_Notations200.csv")

# Ajout de nouvelles colonnes
### Calcul du score humain moyen
df["score_humain"] = df[["Garance", "Matthias", "Yannis"]].mean(axis=1)

# Copie des colonnes d'hallucinations/d'idées invalides en version catégorielle
mapping = {0:"Non", 1:"Oui"}
df["pres_hallu"] = df["Hallucinations"].map(mapping)
df["pres_idees_inv"] = df["Idées_non_ind"].map(mapping)

# Nombre de tokens par contribution et par extractions (comptage simple par espaces)
df["nb_tokens_contrib"] = df["contribution"].apply(lambda x: len(str(x).split()))
df["nb_tokens_extraction"] = df["ideas_text"].apply(lambda x: len(str(x).split()))

# Répartition des contributions en 5 catégories équilibrées selon le nombre de tokens (basé sur les quantiles)
df["contrib_tokens_bins"] = pd.qcut(df["nb_tokens_contrib"], q=5, labels=[
    "Très court", "Court", "Moyen", "Long", "Très long"
])

# Autre options : catégories basées sur des seuils fixes
df["contrib_tokens_bins_fixe"] = pd.cut(df["nb_tokens_contrib"], bins=[0, 50, 100, 200, np.inf], labels=[
    "Très court (0-50)", "Court (51-100)", "Moyen (101-200)", "Long (+200)"
])

# **II - Analyse de la métrique**

In [3]:
# Corrélation de Pearson entre score humain et score QualIT
correlation = df["score_humain"].corr(df["C"])
print(f"Corrélation entre le score humain moyen et le score QualIT : {correlation:.2f}")

Corrélation entre le score humain moyen et le score QualIT : 0.71


La corrélation entre le score humain moyen et la métrique QualIT est assez élevée ***ET*** positive, ce qui suggère qu'elle n'est pas trop mauvaise pour évaluer la qualité de l'extraction. Toutefois, ce résultat doit être interprété avec prudence pour 2 raisons :
- Le désaccord entre évaluateurs humains indique que la notation n'est pas totalement fiable.
- La métrique ne tient pas compte de la pertinence des idées extraites, seulement de leurs similarités avec la contribution originale.

Les extractions échouées ne sont pas sur les graphiques précédents car elles n'ont pas de valeur dans "Hallucinations" ou "Idées_non_ind".

Attention : Les extractions échouées ne sont pas sur la plupart des graphiques car elles n'ont pas de valeur dans "Hallucinations" ou "Idées_non_ind".

In [4]:
# Heatmap de densité en arrière-plan
fig = px.density_contour(
    df, x="C", y="score_humain",
    nbinsx=20, nbinsy=20
)
fig.update_traces(contours_coloring="fill", contours_showlabels = False, colorscale="Blues")
# Scatter plot
fig.add_scatter(
    x=df["C"], y=df["score_humain"],
    mode="markers",
    marker=dict(color="#000000"),
    hovertemplate="Métrique QualIT = %{x}<br>Score humain = %{y}<extra></extra>"
)
fig.add_shape(
    type="line", x0=0, x1=1, y0=0, y1=10,
    xref="x", yref="y",
    line=dict(color="#d41010")
)
fig.update_layout(
    title="<b>Score humain vs métrique QualIT</b>",
    xaxis_title="Métrique QualIT", yaxis_title="Score humain",
    width=700, height=500,
    coloraxis_showscale=False
)
fig.show()

In [6]:
# Coloration avec hallucinations - version linéaire
fig = px.scatter(
    df, x="C", y="score_humain", 
    color="pres_hallu", color_discrete_sequence=["#ff6361", "#003f5c"]
)
fig.add_shape(       # Ajout de la ligne x = y pour référence
    type="line", x0=0, x1=1, y0=0, y1=10, xref="x", yref="y",
    line=dict(color="#d41010")
)
fig.update_layout(
    title={"text": "<b>Score humain vs métrique QualIT</b>"},
    xaxis_title="Métrique QualIT", yaxis_title="Score humain", 
    legend_title="Présence <br>d'hallucinations",
    width=700, height=500
)
fig.update_traces(
    hovertemplate =
        "Métrique QualIT = %{x}<br>" \
        "Score humain = %{y}<extra></extra>"
)
fig.show()

In [7]:
# Coloration avec hallucinations - version logarithmique
df["log_score_humain"] = np.log10(df["score_humain"]+1)

fig = px.scatter(
    df, x="C", y="log_score_humain", 
    color="pres_hallu", color_discrete_sequence=["#ff6361", "#003f5c"]
)
fig.add_shape(      # Ajout de la ligne x = y pour référence
    type="line", x0=0, x1=1, y0=0, y1=1, xref="x", yref="y",
    line=dict(color="#d41010")
)
fig.update_layout(
    title={"text": "<b>log(Score humain + 1) vs métrique QualIT</b>"},
    xaxis_title="Métrique QualIT", yaxis_title="log(Score humain + 1)", 
    legend_title="Présence <br>d'hallucinations",
    width=700, height=500
)
fig.update_traces(
    hovertemplate =
        "Métrique QualIT = %{x}<br>" \
        "log(Score humain + 1) = %{y}<extra></extra>"
)
fig.show()

In [8]:
# Coloration avec hallucinations - version linéaire
fig = px.scatter(
    df, x="C", y="score_humain", 
    color="pres_idees_inv", color_discrete_sequence=["#ff6361", "#003f5c"]
)
fig.add_shape(       # Ajout de la ligne x = y pour référence
    type="line", x0=0, x1=1, y0=0, y1=10, xref="x", yref="y",
    line=dict(color="#d41010")
)
fig.update_layout(
    title={"text": "<b>Score humain vs métrique QualIT</b>"},
    xaxis_title="Métrique QualIT", yaxis_title="Score humain", 
    legend_title="Présence d'idées <br>invalides",
    width=700, height=500
)
fig.update_traces(
    hovertemplate =
        "Métrique QualIT = %{x}<br>" \
        "Score humain = %{y}<extra></extra>"
)
fig.show()

In [9]:
# Version avec les bins équilibrées
fig1 = px.scatter(
    df, x="C", y="score_humain", color="contrib_tokens_bins", 
    color_discrete_sequence=["#ffd380", "#ff6361", "#8a508f", "#003348", "#000000"],
    category_orders={
        "contrib_tokens_bins": ["Très court", "Court", "Moyen", "Long", "Très long"]
    }, 
    symbol="pres_idees_inv", symbol_map={"Oui":"x", "Non":"circle"},
)
fig1.add_shape(       # Ajout de la ligne x = y pour référence
    type="line", x0=0, x1=1, y0=0, y1=10, xref="x", yref="y",
    line=dict(color="#d41010")
)
fig1.update_layout(
    title={"text": "<b>Score humain vs la métrique QualIT</b>"},
    xaxis_title="Métrique QualIT", yaxis_title="Score humain", 
    legend_title="Longueur, présence <br>d'idées invalides",
    width=700, height=500
)
fig1.update_traces(
    hovertemplate =
        "Métrique QualIT = %{x}<br>" \
        "Score humain = %{y}<extra></extra>"
)
# Version avec les bins fixes
fig2 = px.scatter(
    df, x="C", y="score_humain", color="contrib_tokens_bins_fixe", 
    color_discrete_sequence=["#ffd380", "#ff6361", "#8a508f", "#000000"],
    category_orders={
        "contrib_tokens_bins_fixe": ["Très court (0-50)", "Court (51-100)", "Moyen (101-200)", "Long (+200)"]
    },
    symbol="pres_idees_inv", symbol_map={"Oui":"circle", "Non":"x"},
)
fig2.add_shape(       # Ajout de la ligne x = y pour référence
    type="line", x0=0, x1=1, y0=0, y1=10, xref="x", yref="y",
    line=dict(color="#d41010")
)
fig2.update_layout(
    title={"text": "<b>Score humain vs la métrique QualIT</b>"},
    xaxis_title="Métrique QualIT", yaxis_title="Score humain", 
    legend_title="Longueur, présence <br>d'idées invalides",
    width=700, height=500
)
fig2.update_traces(
    hovertemplate =
        "Métrique QualIT = %{x}<br>" \
        "Score humain = %{y}<extra></extra>"
)

# Affichage des figures
fig1.show()
fig2.show()

In [10]:
# Version avec les bins équilibrées
fig1 = px.scatter(
    df, x="C", y="score_humain", color="contrib_tokens_bins", 
    color_discrete_sequence=["#ffd380", "#ff6361", "#8a508f", "#003348", "#000000"],
    category_orders={
        "contrib_tokens_bins": ["Très court", "Court", "Moyen", "Long", "Très long"]
    }, 
    symbol="pres_hallu", symbol_map={"Oui":"x", "Non":"circle"},
)
fig1.add_shape(       # Ajout de la ligne x = y pour référence
    type="line", x0=0, x1=1, y0=0, y1=10, xref="x", yref="y",
    line=dict(color="#d41010")
)
fig1.update_layout(
    title={"text": "<b>Score humain vs la métrique QualIT</b>"},
    xaxis_title="Métrique QualIT", yaxis_title="Score humain", 
    legend_title="Longueur, présence <br>d'hallucinations",
    width=700, height=500
)
fig1.update_traces(
    hovertemplate =
        "Métrique QualIT = %{x}<br>" \
        "Score humain = %{y}<extra></extra>"
)
# Version avec les bins fixes
fig2 = px.scatter(
    df, x="C", y="score_humain", color="contrib_tokens_bins_fixe", 
    color_discrete_sequence=["#ffd380", "#ff6361", "#8a508f", "#000000"],
    category_orders={
        "contrib_tokens_bins_fixe": ["Très court (0-50)", "Court (51-100)", "Moyen (101-200)", "Long (+200)"]
    },
    symbol="pres_hallu", symbol_map={"Oui":"circle", "Non":"x"},
)
fig2.add_shape(       # Ajout de la ligne x = y pour référence
    type="line", x0=0, x1=1, y0=0, y1=10, xref="x", yref="y",
    line=dict(color="#d41010")
)
fig2.update_layout(
    title={"text": "<b>Score humain vs la métrique QualIT</b>"},
    xaxis_title="Métrique QualIT", yaxis_title="Score humain", 
    legend_title="Longueur, présence <br>d'hallucinations",
    width=700, height=500
)
fig2.update_traces(
    hovertemplate =
        "Métrique QualIT = %{x}<br>" \
        "Score humain = %{y}<extra></extra>"
)

# Affichage des figures
fig1.show()
fig2.show()

In [11]:
# Version avec les bins équlibrées
fig1 = px.histogram(
    df, x="contrib_tokens_bins", y="C", histfunc="avg", color="contrib_tokens_bins",
    color_discrete_sequence=["#ffd380", "#ff6361", "#8a508f", "#003348", "#000000"],
    category_orders={
        "contrib_tokens_bins": ["Très court", "Court", "Moyen", "Long", "Très long"]
    }
)
fig1.update_layout(
    title={"text": "<b>Métrique QualIT par catégorie de longueur<b>"}, 
    xaxis_title="Catégories de longueur (en nombre de tokens)", 
    yaxis_title="Moyenne QualIT", 
    width=800, height=450, showlegend=False
)
fig1.update_traces(
    hovertemplate =
        "Catégorie de longueur = %{x}<br>" \
        "Moyenne QualIT = %{y}<extra></extra>"
)
# Version avec les bins fixes
fig2 = px.histogram(
    df, x="contrib_tokens_bins_fixe", y="C", histfunc="avg", color="contrib_tokens_bins_fixe",
    color_discrete_sequence=["#ffd380", "#ff6361", "#8a508f", "#000000"],
    category_orders={
        "contrib_tokens_bins_fixe": ["Très court (0-50)", "Court (51-100)", "Moyen (101-200)", "Long (+200)"]
    }
)
fig2.update_layout(
    title={"text": "<b>Métrique QualIT par catégorie de longueur<b>"}, 
    xaxis_title="Catégories de longueur (en nombre de tokens)", 
    yaxis_title="Moyenne QualIT", 
    width=800, height=450, showlegend=False
)
fig2.update_traces(
    hovertemplate =
        "Catégorie de longueur = %{x}<br>" \
        "Moyenne QualIT = %{y}<extra></extra>"
)

# Affichage des figures
fig1.show()
fig2.show()